In [1]:
import keras
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
from tensorflow.keras.regularizers import l2
import os
import pickle
import time
import matplotlib.pyplot as plt
import tensorflow as tf

#os.environ["OMP_NUM_THREADS"] = '6'
#export TF_ENABLE_ONEDNN_OPTS=1
#export TF_ENABLE_MKL_NATIVE_FORMAT=1
#tf.config.threading.set_inter_op_parallelism_threads(3) 
#tf.config.threading.set_intra_op_parallelism_threads(3)
#tf.config.set_soft_device_placement(enabled = True)
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #to hide tensorflow warnings

2024-02-17 16:24:56.194103: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-17 16:24:57.203382: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#input data function
def load_input_data(input_file):
    with open(input_file, 'rb') as file:
        dataset = pickle.load(file)
        # For our problem we have positions and energy arrays,
        # which are respectively data and labels in common Machine Learning language
        positions_array = np.array([mol['positions'] for mol in dataset])
        energies_array = np.array([mol['energy'] for mol in dataset])

    return positions_array, energies_array

#model creation
def createmodel(input_class, output_class, base_neurons):
    model = Sequential([
        Dense(base_neurons, activation='relu', input_shape=(,input_class), kernel_regularizer=l2(0.01)),
        Dense(base_neurons/2., activation='relu', kernel_regularizer=l2(0.01)),
        Dense(base_neurons/4., activation='gelu', kernel_regularizer=l2(0.01)),
        Dense(base_neurons/8., activation='gelu', kernel_regularizer=l2(0.01)),
        Dense(base_neurons/16., activation='relu', kernel_regularizer=l2(0.01)),
        Dense(output_class)
    ])
    return model

#model compilation
def compilemodel(model):
    #optimizer
    opt = keras.optimizers.Adam() #learning_rate=LR
    #optimizer training with type of dataset
    model.compile(
        #rms
        loss='mean_squared_error',
        #adam with LR and decay
        optimizer=opt,
        #metrics to monitor
        metrics=['mean_squared_error', 'mae']
    )

In [3]:
#define input train and test data directory
#input_file = 'saved_dataset/merged_dataset_rand_5a.pickle'
input_file = 'saved_dataset/merged_randp06_dataset_5a.pickle'
#Load dataset into data and labels (positions=data, energy=labels)
positions_array, energies_array = load_input_data(input_file)

# Splitting the dataset into training, validation, and test sets
# with a 72-20-8% ratio from initial dataset
train_val_data, test_data, train_val_labels, test_labels = train_test_split(
    positions_array, energies_array, test_size=0.2, random_state=42)
# Further splitting train, into new train and validation samples
# Takes 80% and is divided to 10% of that for the validation set
train_data, val_data, train_labels, val_labels = train_test_split(
    train_val_data, train_val_labels, test_size=0.1, random_state=42)

print(train_data.shape[0], 'train samples')
print(test_data.shape[0], 'test samples')
print(val_data.shape[0], 'validation samples')

7272 train samples
2020 test samples
808 validation samples


In [4]:
#define input, output dimensions
input_class = train_data.shape[1] #dH1,dH2,theta, in Angstrom units
output_class = 1 #total system energy in eV
n_iterations = 1
#define learing rate, epochs and batch_size for optimizer and fitter
batch_size = 16
epochs = 4000
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=500)
history = []
neurons = 128
end_neurons = 512

In [5]:
for i in range(0,n_iterations):
    # Base model creation
    #iterations = n_iterations
    #increment = (end_neurons- base_neurons) // (iterations-1)
    #neurons = base_neurons + i*increment
    model = createmodel(input_class, output_class, neurons)
    #neurons = neurons*2
    
    #model name for saving purposes
    ainame = 'conor'
    n_layers = len(model.layers)
    model_name = 'top10bs16_'+str(ainame)+'_'+str(n_layers)+'layers_'+str(neurons)+'neurons_'+'p06cut'
    
    # Model compilation with automatic LR by adam
    compilemodel(model)
    # Model training 
    fitstart = time.time()
    history.append( 
        model.fit(
        train_data, train_labels, epochs=epochs, batch_size=batch_size, 
        validation_data=(val_data, val_labels), 
        callbacks=callback, verbose = 1
             )
                  )
    fitend = time.time()
    print(f'Time passed for fit {i+1}: {fitend-fitstart:.1f} seconds')
    
    # Save model and weights
    #create directory to save weighted model
    save_dir = os.path.join(os.getcwd(), 'saved_model')
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name + '.keras')
    model.save(model_path)
    print('Saved trained model at %s ' % model_path , '\n')
    
    # Estrazione dell'andamento dell'errore sul set di addestramento e sul set di validazione
    train_loss = history[i].history['loss']
    val_loss = history[i].history['val_loss']

    # Creazione del grafico di errore di training e validazione
    plot_dir = os.path.join(os.getcwd(), 'model_plots')
    if not os.path.isdir(plot_dir):
        os.makedirs(plot_dir)
    plot_name = model_name + '__errplot.png'
    plot_path = os.path.join(plot_dir, plot_name)
    
    plt.figure(figsize=(15, 10))
    plt.plot(train_loss, label='Errore sul set di addestramento')
    plt.plot(val_loss, label='Errore sul set di validazione')
    plt.xlabel('Epoche')
    plt.ylabel('Errore')
    plt.ylim(-1, 10)
    plt.grid(axis='both')
    plt.legend()
    plt.title('Andamento dell\'errore')
    plt.savefig(plot_path)

    
    #O_HH plot save
    input_file = 'saved_dataset/dataset_O_HH_100_5a_test.pickle' 

    positions, energies = load_input_data(input_file)
    d1 = positions[:,0] #dH1>dH2
    d2 = positions[:,1]
    theta = positions[:,2]

    input_data = positions
    tf.function(autograph=False, reduce_retracing=True)
    # Effettua previsioni sulle nuove osservazioni
    predictions = model.predict(input_data)

    # Creazione del grafico di O_HH
    graph_dir = os.path.join(os.getcwd(), 'model_graphs_O_HH')
    if not os.path.isdir(graph_dir):
        os.makedirs(graph_dir)
    graph1_name = model_name + '__O_HHfit.png'
    graph1_path = os.path.join(graph_dir, graph1_name)
    
    fig1, ax1 = plt.subplots(figsize=(21.4,12))
    ax1.set(title='Predizione AI per O_HH: 0.1<theta<pi/2 e dH1=dH2= 2A')
    ax1.set(xlabel="Theta [rad]", ylabel="Energy [eV]")
    ax1.scatter(theta, energies, color ='salmon', label='pw.x energies')
    ax1.scatter(theta, predictions, color='darkcyan', label='AI predictions')
    ax1.grid(axis='both')
    ax1.legend(loc='best', ncol=1, facecolor= 'white')
    fig1.savefig(graph1_path, dpi = 200)

    
    #theta plot save
    input_file2 = 'saved_dataset/dataset_theta_100_5a_test.pickle' 

    positions2, energies2 = load_input_data(input_file2)
    d12 = positions2[:,0] #dH1>dH2
    d22 = positions2[:,1]
    theta2 = positions2[:,2]

    input_data2 = positions2
    # Effettua previsioni sulle nuove osservazioni
    predictions2 = model.predict(input_data2)

    # Creazione del grafico di theta
    graph2_dir = os.path.join(os.getcwd(), 'model_graphs_theta')
    if not os.path.isdir(graph2_dir):
        os.makedirs(graph2_dir)
    graph2_name = model_name + '__thetafit.png'
    graph2_path = os.path.join(graph2_dir, graph2_name)
    
    fig2, ax2 = plt.subplots(figsize=(21.4,12))
    ax2.set(title='Predizione AI per 1<theta<pi e dH1=dH2= 0.96A')
    ax2.set(xlabel="Theta [rad]", ylabel="Energy [eV]")
    ax2.scatter(theta2, energies2, color ='salmon', label='pw.x energies')
    ax2.scatter(theta2, predictions2, color='darkcyan', label='AI predictions')
    ax2.grid(axis='both')
    ax2.legend(loc='best', ncol=1, facecolor= 'white')
    fig2.savefig(graph2_path, dpi = 200)

    
    #dh2 plot save
    input_file3 = 'saved_dataset/dataset_dh2_100_5a_test.pickle' 

    positions3, energies3 = load_input_data(input_file3)
    d13 = positions3[:,0] #dH1>dH2
    d23 = positions3[:,1]
    theta3 = positions3[:,2]

    input_data3 = positions3
    # Effettua previsioni sulle nuove osservazioni
    predictions3 = model.predict(input_data3)

    # Creazione del grafico di theta
    graph3_dir = os.path.join(os.getcwd(), 'model_graphs_dh2')
    if not os.path.isdir(graph3_dir):
        os.makedirs(graph3_dir)
    graph3_name = model_name + '__dh2fit.png'
    graph3_path = os.path.join(graph3_dir, graph3_name)
    
    fig3, ax3 = plt.subplots(figsize=(21.4,12))
    ax3.set(title='Predizione AI per 0.6<dH2<3.6, dH1=0.96A, theta=1.82rad')
    ax3.set(xlabel="dH2 [Angstrom]", ylabel="Energy [eV]")
    ax3.scatter(d23, energies3, color ='salmon', label='pw.x energies')
    ax3.scatter(d23, predictions3, color='darkcyan', label='AI predictions')
    ax3.grid(axis='both')
    ax3.legend(loc='best', ncol=1, facecolor= 'white')
    fig3.savefig(graph3_path, dpi = 200)
    
    #OH_H plot save
    input_file4 = 'saved_dataset/dataset_OH_H_100_5a_test.pickle' 

    positions4, energies4 = load_input_data(input_file4)
    d14 = positions4[:,0] #dH1>dH2
    d24 = positions4[:,1]
    theta4 = positions4[:,2]

    input_data4 = positions4
    # Effettua previsioni sulle nuove osservazioni
    predictions4 = model.predict(input_data4)

    # Creazione del grafico di OH_H
    graph4_dir = os.path.join(os.getcwd(), 'model_graphs_OH_H')
    if not os.path.isdir(graph4_dir):
        os.makedirs(graph4_dir)
    graph4_name = model_name + '__OH_Hfit.png'
    graph4_path = os.path.join(graph4_dir, graph4_name)
    
    fig4, ax4 = plt.subplots(figsize=(21.4,12))
    ax4.set(title='Predizione AI per OH_H: 0.6<dH1<2, theta=1.82rad e dH2= 3A')
    ax4.set(xlabel="dH1 [Angstrom]", ylabel="Energy [eV]")
    ax4.scatter(d14, energies4, color ='salmon', label='pw.x energies')
    ax4.scatter(d14, predictions4, color='darkcyan', label='AI predictions')
    ax4.grid(axis='both')
    ax4.legend(loc='best', ncol=1, facecolor= 'white')
    fig4.savefig(graph4_path, dpi = 200)  
    
        #OH_H plot save
    input_file5 = 'saved_dataset/dataset_O_H_H_100_5a_test.pickle' 

    positions5, energies5 = load_input_data(input_file5)
    d15 = positions5[:,0] #dH1>dH2
    d25 = positions5[:,1]
    theta5 = positions5[:,2]

    input_data5 = positions5
    # Effettua previsioni sulle nuove osservazioni
    predictions5 = model.predict(input_data5)

    # Creazione del grafico di OH_H
    graph5_dir = os.path.join(os.getcwd(), 'model_graphs_O_H_H')
    if not os.path.isdir(graph5_dir):
        os.makedirs(graph5_dir)
    graph5_name = model_name + '__O_H_Hfit.png'
    graph5_path = os.path.join(graph5_dir, graph5_name)
    
    fig5, ax5 = plt.subplots(figsize=(21.4,12))
    ax5.set(title='Predizione AI per O_H_H: 0.5<dH1<3.5 A, theta=0.37 rad e dH2= 2A')
    ax5.set(xlabel="dH1 [Angstrom]", ylabel="Energy [eV]")
    ax5.scatter(d15, energies5, color ='salmon', label='pw.x energies')
    ax5.scatter(d15, predictions5, color='darkcyan', label='AI predictions')
    ax5.grid(axis='both')
    ax5.legend(loc='best', ncol=1, facecolor= 'white')
    fig5.savefig(graph5_path, dpi = 200)  
    
    plt.close('all')
    
    #duration = 3  # seconds
    #freq = 440  # Hz
    #os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))
    print('\007')

Epoch 1/4000


2024-02-17 16:25:01.066223: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


455/455 [==============================] - 2s 2ms/step - loss: 47769.6836 - mean_squared_error: 47767.8711 - mae: 152.7442 - val_loss: 5804.0918 - val_mean_squared_error: 5802.0117 - val_mae: 62.5661
Epoch 2/4000
455/455 [==============================] - 0s 1ms/step - loss: 2316.4634 - mean_squared_error: 2313.8323 - mae: 34.4629 - val_loss: 48.5024 - val_mean_squared_error: 45.3764 - val_mae: 5.0195
Epoch 3/4000
455/455 [==============================] - 0s 1ms/step - loss: 17.7823 - mean_squared_error: 14.6270 - mae: 2.4418 - val_loss: 15.4775 - val_mean_squared_error: 12.3294 - val_mae: 2.5864
Epoch 4/4000
455/455 [==============================] - 0s 982us/step - loss: 10.2940 - mean_squared_error: 7.1524 - mae: 1.6557 - val_loss: 12.3985 - val_mean_squared_error: 9.2661 - val_mae: 1.4518
Epoch 5/4000
455/455 [==============================] - 0s 991us/step - loss: 9.2988 - mean_squared_error: 6.1808 - mae: 1.5323 - val_loss: 11.8369 - val_mean_squared_error: 8.7304 - val_mae: 1.3

455/455 [==============================] - 0s 974us/step - loss: 6.9290 - mean_squared_error: 4.5775 - mae: 1.4196 - val_loss: 8.2284 - val_mean_squared_error: 5.8839 - val_mae: 1.2630
Epoch 43/4000
455/455 [==============================] - 0s 958us/step - loss: 7.3922 - mean_squared_error: 5.0525 - mae: 1.5412 - val_loss: 7.9218 - val_mean_squared_error: 5.5893 - val_mae: 1.0668
Epoch 44/4000
455/455 [==============================] - 0s 950us/step - loss: 7.0764 - mean_squared_error: 4.7505 - mae: 1.4836 - val_loss: 13.3540 - val_mean_squared_error: 11.0369 - val_mae: 2.7658
Epoch 45/4000
455/455 [==============================] - 0s 962us/step - loss: 7.0526 - mean_squared_error: 4.7372 - mae: 1.4965 - val_loss: 10.7353 - val_mean_squared_error: 8.4267 - val_mae: 2.2729
Epoch 46/4000
455/455 [==============================] - 0s 961us/step - loss: 6.5838 - mean_squared_error: 4.2791 - mae: 1.3595 - val_loss: 8.8736 - val_mean_squared_error: 6.5810 - val_mae: 1.6918
Epoch 47/4000
45

455/455 [==============================] - 0s 943us/step - loss: 5.8566 - mean_squared_error: 4.1743 - mae: 1.3336 - val_loss: 7.0434 - val_mean_squared_error: 5.3625 - val_mae: 1.2104
Epoch 125/4000
455/455 [==============================] - 0s 957us/step - loss: 6.0868 - mean_squared_error: 4.4121 - mae: 1.4326 - val_loss: 10.2618 - val_mean_squared_error: 8.5929 - val_mae: 2.2285
Epoch 126/4000
455/455 [==============================] - 0s 957us/step - loss: 7.5305 - mean_squared_error: 5.8604 - mae: 1.6182 - val_loss: 8.1148 - val_mean_squared_error: 6.4492 - val_mae: 1.6275
Epoch 127/4000
455/455 [==============================] - 0s 944us/step - loss: 6.0078 - mean_squared_error: 4.3424 - mae: 1.4097 - val_loss: 9.1506 - val_mean_squared_error: 7.4859 - val_mae: 1.6514
Epoch 128/4000
455/455 [==============================] - 0s 958us/step - loss: 5.5825 - mean_squared_error: 3.9207 - mae: 1.3086 - val_loss: 8.9771 - val_mean_squared_error: 7.3109 - val_mae: 1.8431
Epoch 129/4000

...

Epoch 3814/4000
455/455 [==============================] - 0s 975us/step - loss: 1.6952 - mean_squared_error: 1.2917 - mae: 0.6692 - val_loss: 3.8702 - val_mean_squared_error: 3.4661 - val_mae: 0.6952
Epoch 3815/4000
455/455 [==============================] - 0s 965us/step - loss: 1.8613 - mean_squared_error: 1.4571 - mae: 0.7396 - val_loss: 3.8127 - val_mean_squared_error: 3.4081 - val_mae: 0.6810
Epoch 3816/4000
455/455 [==============================] - 0s 963us/step - loss: 1.7849 - mean_squared_error: 1.3801 - mae: 0.7073 - val_loss: 3.8440 - val_mean_squared_error: 3.4403 - val_mae: 0.7480
Epoch 3817/4000
455/455 [==============================] - 0s 977us/step - loss: 1.8564 - mean_squared_error: 1.4524 - mae: 0.7531 - val_loss: 3.8408 - val_mean_squared_error: 3.4363 - val_mae: 0.6407
Epoch 3818/4000
455/455 [==============================] - 0s 985us/step - loss: 1.6777 - mean_squared_error: 1.2727 - mae: 0.6594 - val_loss: 3.8592 - val_mean_squared_error: 3.4541 - val_mae: 0.

Epoch 3855/4000
455/455 [==============================] - 0s 941us/step - loss: 1.7546 - mean_squared_error: 1.3503 - mae: 0.6843 - val_loss: 3.7892 - val_mean_squared_error: 3.3846 - val_mae: 0.7129
Epoch 3856/4000
455/455 [==============================] - 0s 951us/step - loss: 1.7500 - mean_squared_error: 1.3455 - mae: 0.7017 - val_loss: 4.1577 - val_mean_squared_error: 3.7540 - val_mae: 0.8112
Epoch 3857/4000
455/455 [==============================] - 0s 991us/step - loss: 1.7026 - mean_squared_error: 1.2980 - mae: 0.6478 - val_loss: 3.6899 - val_mean_squared_error: 3.2853 - val_mae: 0.5807
Epoch 3858/4000
455/455 [==============================] - 0s 977us/step - loss: 1.7191 - mean_squared_error: 1.3143 - mae: 0.6815 - val_loss: 4.4267 - val_mean_squared_error: 4.0224 - val_mae: 0.8674
Epoch 3859/4000
455/455 [==============================] - 0s 958us/step - loss: 1.6774 - mean_squared_error: 1.2733 - mae: 0.6537 - val_loss: 3.9090 - val_mean_squared_error: 3.5052 - val_mae: 0.

Epoch 3896/4000
455/455 [==============================] - 0s 973us/step - loss: 1.7261 - mean_squared_error: 1.3260 - mae: 0.6668 - val_loss: 6.4250 - val_mean_squared_error: 6.0247 - val_mae: 1.8851
Epoch 3897/4000
455/455 [==============================] - 0s 964us/step - loss: 1.7169 - mean_squared_error: 1.3154 - mae: 0.6603 - val_loss: 4.1236 - val_mean_squared_error: 3.7220 - val_mae: 0.7891
Epoch 3898/4000
455/455 [==============================] - 0s 964us/step - loss: 1.7587 - mean_squared_error: 1.3575 - mae: 0.6889 - val_loss: 4.0314 - val_mean_squared_error: 3.6297 - val_mae: 0.7227
Epoch 3899/4000
455/455 [==============================] - 0s 966us/step - loss: 1.7431 - mean_squared_error: 1.3419 - mae: 0.6691 - val_loss: 4.6970 - val_mean_squared_error: 4.2950 - val_mae: 1.1489
Epoch 3900/4000
455/455 [==============================] - 0s 968us/step - loss: 1.7457 - mean_squared_error: 1.3445 - mae: 0.6776 - val_loss: 4.8911 - val_mean_squared_error: 4.4895 - val_mae: 1.

Epoch 3937/4000
455/455 [==============================] - 0s 1ms/step - loss: 1.8309 - mean_squared_error: 1.4304 - mae: 0.7168 - val_loss: 3.9390 - val_mean_squared_error: 3.5383 - val_mae: 0.7653
Epoch 3938/4000
455/455 [==============================] - 0s 954us/step - loss: 1.6290 - mean_squared_error: 1.2282 - mae: 0.6302 - val_loss: 3.9427 - val_mean_squared_error: 3.5423 - val_mae: 0.7161
Epoch 3939/4000
455/455 [==============================] - 0s 946us/step - loss: 1.7103 - mean_squared_error: 1.3085 - mae: 0.6662 - val_loss: 3.8317 - val_mean_squared_error: 3.4305 - val_mae: 0.5908
Epoch 3940/4000
455/455 [==============================] - 0s 951us/step - loss: 1.7022 - mean_squared_error: 1.3018 - mae: 0.6713 - val_loss: 3.7027 - val_mean_squared_error: 3.3021 - val_mae: 0.6827
Epoch 3941/4000
455/455 [==============================] - 0s 972us/step - loss: 1.7270 - mean_squared_error: 1.3265 - mae: 0.6738 - val_loss: 4.4670 - val_mean_squared_error: 4.0679 - val_mae: 1.08

Epoch 3978/4000
455/455 [==============================] - 0s 972us/step - loss: 1.7542 - mean_squared_error: 1.3581 - mae: 0.6791 - val_loss: 4.6918 - val_mean_squared_error: 4.2956 - val_mae: 1.2691
Epoch 3979/4000
455/455 [==============================] - 0s 945us/step - loss: 1.6645 - mean_squared_error: 1.2667 - mae: 0.6648 - val_loss: 4.0330 - val_mean_squared_error: 3.6354 - val_mae: 0.7543
Epoch 3980/4000
455/455 [==============================] - 0s 966us/step - loss: 1.6578 - mean_squared_error: 1.2605 - mae: 0.6437 - val_loss: 3.7989 - val_mean_squared_error: 3.4021 - val_mae: 0.6133
Epoch 3981/4000
455/455 [==============================] - 0s 991us/step - loss: 1.7966 - mean_squared_error: 1.3998 - mae: 0.7203 - val_loss: 4.9674 - val_mean_squared_error: 4.5690 - val_mae: 1.1907
Epoch 3982/4000
455/455 [==============================] - 0s 1ms/step - loss: 1.7285 - mean_squared_error: 1.3302 - mae: 0.6905 - val_loss: 3.7615 - val_mean_squared_error: 3.3622 - val_mae: 0.60

battle royale di modelli

cambio anche modelli generati automaticamente: numero di layer, numero di neuroni per layer...
    design a cono
 fino a quando non trovo i top 10 modelli- addestramento con 1000 samples

1-battleroyale_conorelu_5layers_512neurons_10k


poi su quei 10 cambio parametri di LR, epochs, batch size
e vedo qual e il migliore e ne tengo solo 1

1 ciclo for per i parametri
1 ciclo per il training a parametri fissati

l-ultimo modello che va meglio posso provare senza random state